<a href="https://colab.research.google.com/github/lakshaygola/TensorFlow-And-DeepLearning/blob/main/CNN_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network (CNN)

This architecture of neural network used for image classification or to analyze visual imagery. In this we  have a different kernel which help us to achive different outcome from the image and when we train the CNN we usually changing the values of the kernel so that we get the better outcome from the model

In this particular example we are using Fashion MNIST dataset which was easily available on kaggle but we are using built in dataset in keras.


In [1]:
pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 4.9MB/s 


In [2]:
# Importing the necessary libraries 
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
from tensorflow.keras.models import Model
import keras.layers as layer
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

import matplotlib.pyplot as plt

## Dataset

In this dataset we have different images of the clothes and they all classified in some different classes.

In [3]:
# Loading the dataset
image_data = keras.datasets.fashion_mnist

In [4]:
# Making training and test data split
(train_image, train_label),  (test_image, test_label) = image_data.load_data() 

4423680/4422102 [==============================] - 0s 0us/step


# Preprocessing of the images

1. Normalizing the image of the dataset

2. Adding color channel in the shape of the image

In [5]:
# Normalizing the values present (ranging the values between 0 and 1)
train_image = train_image / 255.0
test_image = test_image / 255.0

In [6]:
# Cheaking the shape of the image 
print(train_image[0].shape)

# Reshaping the images of train set and test set
train_image = train_image.reshape(len(train_image), 28, 28, 1)
test_image = test_image.reshape(len(test_image), 28, 28, 1)       # Adding one color channel because dataset contain gray scale image
print(train_image[0].shape)

(28, 28)
(28, 28, 1)


# Making the model 

In [7]:
# Model building function and using keras tuner
def build_model(hp):
  model= keras.Sequential([
          # 1st layer
          layer.Conv2D(
              filters= hp.Int('conv2d_filter', min_value = 32, max_value = 512, step = 16),
              kernel_size = hp.Choice('kernel_size', values = [3, 5]),
              activation= 'relu',
              input_shape= (28,28,1)
          ),

          # 2nd layer
          layer.Conv2D(
              filters= hp.Int('conv2d_filter', min_value = 32, max_value = 128, step = 16),
              kernel_size = hp.Choice('kernel_size', values = [3, 5]),
              activation = 'relu'
          ),

          # 3rd layer
          layer.Conv2D(
              filters= hp.Int('conv2d_filter', min_value = 32, max_value = 64, step = 16),
              kernel_size = hp.Choice('kernel_size', values = [3, 5]),
              activation = 'relu'
          ), 
          # Flattern layer
          layer.Flatten(), 
          # 1st Dense layer
          layer.Dense(units = hp.Int('unit_num', min_value = 32, max_value = 64, step = 16), 
                      activation = 'relu'),
          # 2nd Dense layer
          layer.Dense(10, activation= 'sigmoid')
  ])

  model.compile(optimizer= keras.optimizers.Adam(hp.Choice('learning_rate', values = [0.001, 0.0001])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

# Searching best pair of hyperparameter for the model

In [8]:
# Searching for the best values of hyperparameter for our model
tuner = RandomSearch(build_model, 
                     objective = 'val_accuracy', 
                     max_trials = 2, 
                     directory= 'model_output', 
                     project_name = 'fashion_model')

In [9]:
# Perform searching for the different set to hyperparameters
tuner.search(train_image, train_label, validation_split = 0.3, epochs = 3)

Trial 2 Complete [00h 01m 57s]
val_accuracy: 0.9021111130714417

Best val_accuracy So Far: 0.9042222499847412
Total elapsed time: 00h 09m 00s
INFO:tensorflow:Oracle triggered exit


In [12]:
# Making model using best values for the hyper parameters
model = tuner.get_best_models(num_models = 1)[0]

In [13]:
# Taking the summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 416)       4160      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 416)       1557920   
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 416)       1557920   
_________________________________________________________________
flatten (Flatten)            (None, 201344)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                12886080  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 16,006,730
Trainable params: 16,006,730
Non-trainable params: 0
____________________________________________

## Training the model


In [14]:
model.fit(train_image, train_label, epochs= 10, validation_split= 0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 153s 90ms/step - loss: 0.2211 - accuracy: 0.9202 - val_loss: 0.2613 - val_accuracy: 0.9087
Epoch 5/10
1688/1688 [==============================] - 152s 90ms/step - loss: 0.1591 - accuracy: 0.9400 - val_loss: 0.2567 - val_accuracy: 0.9140
Epoch 6/10
1688/1688 [==============================] - 152s 90ms/step - loss: 0.1126 - accuracy: 0.9577 - val_loss: 0.2645 - val_accuracy: 0.9100
Epoch 7/10
1688/1688 [==============================] - 152s 90ms/step - loss: 0.0746 - accuracy: 0.9736 - val_loss: 0.3094 - val_accuracy: 0.9172
Epoch 8/10
1688/1688 [==============================] - 152s 90ms/step - loss: 0.0484 - accuracy: 0.9824 - val_loss: 0.3708 - val_accuracy: 0.9167
Epoch 9/10
1688/1688 [==============================] - 152s 90ms/step - loss: 0.0379 - accuracy: 0.9864 - val_loss: 0.4121 - val_accuracy: 0.9155
Epoch 10/10
1688/1688 [==============================] - 152s 90ms/step - loss: 0.0327 - accuracy: 0.9882 - val_loss: 